<a href="https://colab.research.google.com/github/SocTrep/Investigations/blob/main/GuyanaGoldMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap retry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.6 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [4]:
ee.Authenticate()

In [11]:
ee.Initialize(project='illegal-gold-mining-in-guyana', opt_url="https://earthengine-highvolume.googleapis.com")

In [20]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [21]:
# region = Map.user_roi
region = ee.Geometry.Polygon(
    [
        [
            [-122.513695, 37.707998],
            [-122.513695, 37.804359],
            [-122.371902, 37.804359],
            [-122.371902, 37.707998],
            [-122.513695, 37.707998],
        ]
    ],
    None,
    False,
)

In [22]:
image = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(region)
    .filterDate("2020", "2021")
    .mosaic()
    .clip(region)
    .select("N", "R", "G")
)

In [23]:
# image = ee.ImageCollection('COPERNICUS/S2_SR') \
#             .filterBounds(region) \
#             .filterDate('2021', '2022') \
#             .select('B8', 'B4', 'B3') \
#             .median() \
#             .visualize(min=0, max=4000) \
#             .clip(region)

In [24]:
params = {
    "count": 100,  # How many image chips to export > this corresponds to the below
    #code generating a list of work items (def getRequest)
    "buffer": 127,  # The buffer distance (m) around each point
    "scale": 100,  # The scale to do stratified sampling
    "seed": 1,  # A randomization seed to use for subsampling.
    "dimensions": "256x256",  # The dimension of each image chip
    "format": "png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    "prefix": "tile_",  # The filename prefix
    "processes": 25,  # How many processes to used for parallel processing
    "out_dir": ".",  # The output directory. Default to the current working directly
}

In [25]:
Map.addLayer(image, {}, "Image")
Map.addLayer(region, {}, "ROI", False)
Map.setCenter(-122.4415, 37.7555, 12)
Map

Map(bottom=624.0, center=[37.7555, -122.4415], controls=(WidgetControl(options=['position', 'transparent_bg'],…

In [26]:
def getRequests():
    img = ee.Image(1).rename("Class").addBands(image)
    points = img.stratifiedSample(
        numPoints=params["count"],
        region=region,
        scale=params["scale"],
        seed=params["seed"],
        geometries=True,
    )
    Map.data = points
    return points.aggregate_array(".geo").getInfo()

In [27]:
#The getResult() function then takes one of those points and generates an image centered on that location

@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    point = ee.Geometry.Point(point["coordinates"])
    region = point.buffer(params["buffer"]).bounds()

    if params["format"] in ["png", "jpg"]:
        url = image.getThumbURL(
            {
                "region": region,
                "dimensions": params["dimensions"],
                "format": params["format"],
            }
        )
    else:
        url = image.getDownloadURL(
            {
                "region": region,
                "dimensions": params["dimensions"],
                "format": params["format"],
            }
        )

    if params["format"] == "GEO_TIFF":
        ext = "tif"
    else:
        ext = params["format"]

    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params["out_dir"])
    basename = str(index).zfill(len(str(params["count"])))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, "wb") as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

In [29]:
%%time
logging.basicConfig()
items = getRequests()

pool = multiprocessing.Pool(params["processes"])
pool.starmap(getResult, enumerate(items))

pool.close()

Done:  009
Done:  003
Done:  002
Done:  021
Done:  Done:  000
024Done:  019Done: 

 022
Done:  006
Done:  016
Done:  014
Done:  015
Done:  005
Done:  012Done: 
 023
Done:  026
Done:  001
Done:  025
Done:  008
Done:  028
Done:  029
Done:  018
Done: Done:   011027

Done:  007
Done:  031Done: 
 017
Done:  030
Done:  035
Done:  020
Done:  043
Done:  004
Done:  046Done: 
 013
Done:  048
Done:  044
Done: Done:   039042
Done: 
 Done: Done: 041 
Done: 051 Done: Done:  
050 Done:  Done: 040
Done: 
Done: 034Done: 036 010 032  
Done: Done: 
Done: 
 
049 Done: 047038 045 

 052
033
054


Done:  053
Done:  Done: 037 
Done:  059057Done:  

056
Done:  058
Done:  055
Done:  064
Done:  061
Done:  060
Done:  065
Done: Done:  063 067

Done:  068
Done:  077
Done: Done:   069078Done: 

 072
Done:  070
Done:  071
Done:  073Done: 
066 
Done:  062
Done:  Done: 
074 075
Done:  076
Done:  080
Done:  085
Done:  Done: 082 Done: 
090 081

Done:  087
Done:  Done: 089 Done:  079
088

Done:  084
Done:  086
Done:  092

In [30]:
Map.addLayer(Map.data, {}, "Sample points")
Map

Map(bottom=810790.0, center=[37.788895345521794, -122.49189376831056], controls=(WidgetControl(options=['posit…

In [31]:
#now pull Guyana data

guyMap = geemap.Map()
guyMap

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [35]:
region = guyMap.user_roi

In [36]:
region.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-59.554825, 5.236606],
   [-59.580917, 5.268743],
   [-59.563065, 5.272162],
   [-59.508133, 5.302245],
   [-59.495773, 5.297459],
   [-59.517059, 5.278315],
   [-59.50882, 5.263957],
   [-59.48822, 5.279683],
   [-59.448395, 5.287887],
   [-59.422989, 5.306348],
   [-59.410629, 5.299511],
   [-59.39621, 5.314552],
   [-59.387283, 5.302929],
   [-59.38591, 5.27011],
   [-59.39827, 5.262589],
   [-59.391403, 5.2537],
   [-59.367371, 5.228401],
   [-59.370117, 5.216092],
   [-59.37973, 5.214725],
   [-59.366684, 5.200365],
   [-59.376984, 5.199681],
   [-59.366684, 5.174379],
   [-59.389343, 5.161386],
   [-59.389343, 5.123088],
   [-59.37149, 5.117617],
   [-59.363251, 5.079317],
   [-59.39003, 5.043751],
   [-59.403076, 5.046487],
   [-59.410629, 5.039647],
   [-59.474487, 5.083421],
   [-59.521866, 5.101887],
   [-59.528046, 5.09368],
   [-59.519119, 5.082737],
   [-59.534225, 5.074529],
   [-59.545898, 5.080685],
   [-59.5390